# libs 

In [1]:
import pandas as pd
import selenium as sl 
import time 

# PIPE

## 1 - Coleta e Identificação da Origem dos Dados

### 1.1 Recebimento do arquivo de origem

In [ ]:
# checa a apsta e se achar um arquivo, pega o path extençaõ para usar, SE NAO TIVER AS EXTENSAO ACEITA RETORNA ERRO

file_path = 'ESTOQUE MASUQUERADE.XLSX'
file_ext = 'XSLX'

### 1.2 Classificação da Planilha (Tipo e Finalidade)


In [ ]:
if file_ext == 'XSLX' or 'CSV':
    df_prod_main = pd.read_excel(file_path, skiprows=1)
elif file_ext == 'XML':
    print("xml")
    # converter para DataFrame, extraindo apenas os campos úteis